In [1]:
import pandas as pd
import os
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import plot, iplot, init_notebook_mode
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
init_notebook_mode(connected=True)
cf.go_offline()

# Timeseries Sentiment

## Confirmed Cases

In [124]:
daily_cases = pd.read_excel(f"{parent_path}/Analysis/Sentiment Analysis_Topic Modelling/Covid-19 SG.xlsx") 
daily_cases.reset_index(inplace=True)
daily_cases

daily_confirmed = daily_cases.iloc[:, 1:3]
daily_confirmed

,Date,Daily Confirmed
0,2020-01-23,1
1,2020-01-24,2
2,2020-01-25,1
3,2020-01-26,0
4,2020-01-27,1
...,...,...
415,2021-03-13,8
416,2021-03-14,17
417,2021-03-15,12
418,2021-03-16,11


In [3]:
def sentiment_timeseries(policy, timeframe):
    df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_opinions.csv")

    # Standardising datetime format
    dates = []
    for row in df['Comment Datetime']:
        if '/' in row:
            if '/' in row[-4:]:
                date = dt.datetime.strptime(row, "%d/%m/%y")
            else: 
                date = dt.datetime.strptime(row, "%d/%m/%Y")
            dates.append(date.date())
        elif '-' in row:
            date = dt.datetime.strptime(row, "%Y-%m-%d")
            dates.append(date.date())

    df['Comment Datetime'] = dates

    # Truncate according to the policy timeframe
    start, end = pd.to_datetime([timeframe[0], timeframe[1]], format='%d%b%Y')
    df_within_date = df[(df['Comment Datetime'] >= start) & (df['Comment Datetime'] <= end)]
    df_within_date = df_within_date[['Comment Datetime', 'Vader_compound_score']]

    # Finding the mean vader sentiment score per day 
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Plotting the figure 
    fig = go.Figure(layout_title_text=f"Timeseries Sentiment Analysis of {policy}")
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Rolling 7-Day Average of Vader Compound Score"))
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", visible='legendonly'))
    fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1))
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Vader Compound Score')

    fig.write_image(f"timeseries sentiment plots/{policy}.png", engine="kaleido")
    fig.write_html(f"timeseries sentiment plots/{policy}.html")
    #fig.show()
    print(policy, "is done!")


In [135]:
parent_path = "/Users/joshuawong/Documents/GitHub/Covid-19-Singapore-Analysis"

policy_list = ["circuit breaker", "economic measures", "foreign worker", "mask", "social distancing", "tracetogether", "vaccination"]

policy_timeframe = {"circuit breaker":("1APR2020", "1JUL2020"), 
                    "economic measures":("1FEB2020", "1MAR2021"), 
                    "foreign worker":("1APR2020","1JAN2021"), 
                    "mask":("1FEB2020","1MAR2021"), 
                    "social distancing":("1FEB2020", "1FEB2021"), 
                    "tracetogether":("1MAR2020", "1MAR2021"), 
                    "vaccination":("1AUG2020","1MAR2021")}

for policy, timeframe in policy_timeframe.items():
    sentiment_timeseries(policy, timeframe)
    # break


KeyError: 'Daily Confirmed'

In [96]:
def sentiment_timeseries(policy, timeframe):
    df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_opinions.csv")

    # Standardising datetime format
    dates = []
    for row in df['Comment Datetime']:
        if '/' in row:
            if '/' in row[-4:]:
                date = dt.datetime.strptime(row, "%d/%m/%y")
            else: 
                date = dt.datetime.strptime(row, "%d/%m/%Y")
            dates.append(date.date())
        elif '-' in row:
            date = dt.datetime.strptime(row, "%Y-%m-%d")
            dates.append(date.date())

    df['Comment Datetime'] = dates

    # Truncate according to the policy timeframe
    start, end = pd.to_datetime([timeframe[0], timeframe[1]], format='%d%b%Y')
    df_within_date = df[(df['Comment Datetime'] >= start) & (df['Comment Datetime'] <= end)]
    df_within_date = df_within_date[['Comment Datetime', 'Vader_compound_score']]

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Finding the number of comments per day
    grouped_date_counts = df_within_date.groupby(by=["Comment Datetime"]).count()
    grouped_date_counts.reset_index(inplace=True)
    print(grouped_date_counts)
    grouped_date['Comment Count'] = grouped_date_counts['Vader_compound_score']

    # print(grouped_date)

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", visible='legendonly', opacity=.3), secondary_y=False)

    fig.add_bar(x = grouped_date['Comment Datetime'], y = grouped_date['Comment Count'],
                name="Comment Count", secondary_y=True
                # , visible='legendonly'
                )

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy}",
                      yaxis_range=[-1,1])

    max_num_comments = grouped_date['Comment Count'].max()
    fig.update_layout(yaxis2_range=[0,max_num_comments*2.5])
    fig.update_layout(width = 1200, height=800)
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Vader Compound Score')
    fig.update_yaxes(title_text='Comment Count', secondary_y=True)

    fig.write_image(f"timeseries sentiment plots/{policy}.png", engine="kaleido")
    # fig.write_html(f"timeseries sentiment plots/{policy}.html")
    fig.show()
    print(policy, "is done!")

In [148]:
def sentiment_timeseries_with_cases(policy, timeframe):
    df = pd.read_csv(f"{parent_path}/Data/Sentiment Data/{policy}_valuable_opinions.csv")

    # Standardising datetime format
    dates = []
    for row in df['Comment Datetime']:
        if '/' in row:
            if '/' in row[-4:]:
                date = dt.datetime.strptime(row, "%d/%m/%y")
            else: 
                date = dt.datetime.strptime(row, "%d/%m/%Y")
            dates.append(date.date())
        elif '-' in row:
            date = dt.datetime.strptime(row, "%Y-%m-%d")
            dates.append(date.date())

    df['Comment Datetime'] = dates

    # Truncate according to the policy timeframe
    start, end = pd.to_datetime([timeframe[0], timeframe[1]], format='%d%b%Y')
    df_within_date = df[(df['Comment Datetime'] >= start) & (df['Comment Datetime'] <= end)]
    df_within_date = df_within_date[['Comment Datetime', 'Vader_compound_score']]

    # Finding the mean vader sentiment score per day
    grouped_date = df_within_date.groupby(by=["Comment Datetime"]).mean()
    grouped_date.reset_index(inplace=True)

    # Finding the moving average (7-day)
    grouped_date['SMA_7'] = grouped_date.Vader_compound_score.rolling(7, min_periods=1).mean()

    # Finding the number of comments per day
    grouped_date_counts = df_within_date.groupby(by=["Comment Datetime"]).count()
    grouped_date_counts.reset_index(inplace=True)
    # print(grouped_date_counts)
    grouped_date['Comment Count'] = grouped_date_counts['Vader_compound_score']
    # print(grouped_date)

    daily_confirmed_within_timeframe = daily_confirmed.loc[(daily_confirmed["Date"]>= timeframe[0]) & (daily_confirmed["Date"]<= timeframe[1])]

    # Plotting the figure 
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # NEW
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['SMA_7'], mode='lines', 
                  name="Vader 7-day MA"), secondary_y=False)
    fig.add_trace(go.Scatter(x = grouped_date['Comment Datetime'], y = grouped_date['Vader_compound_score'], mode='lines', 
                  name="Vader Compound Score", opacity=.5), secondary_y=False)

    fig.add_bar(x = daily_confirmed_within_timeframe["Date"], y = daily_confirmed_within_timeframe["Daily Confirmed "],
                name="Cases Count", secondary_y=True
                # , visible='legendonly'
                )

    #fig.update_layout(width = 1200, height=800)
    fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                      title_text=f"Timeseries Sentiment Analysis of {policy}",
                      yaxis_range=[-2,1],
                      yaxis2_range=[0,1426*2])

    fig.update_xaxes(title_text='Date', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Vader Compound Score', ticks="outside", showgrid=False)
    fig.update_yaxes(title_text='Number of Cases', ticks="outside", showgrid=False, secondary_y=True)

    # fig.write_image(f"timeseries sentiment plots/{policy}.png", engine="kaleido")
    # fig.write_html(f"timeseries sentiment plots/{policy}.html")
    fig.show()
    print(policy, "is done!")

In [147]:
parent_path = "/Users/joshuawong/Documents/GitHub/Covid-19-Singapore-Analysis"

policy_list = ["circuit breaker", "economic measures", "foreign worker", "mask", "social distancing", "tracetogether", "vaccination"]

policy_timeframe = {"circuit breaker":("1APR2020", "1JUL2020"), 
                    "economic measures":("1FEB2020", "1MAR2021"), 
                    "foreign worker":("1APR2020","1JAN2021"), 
                    "mask":("1FEB2020","1MAR2021"), 
                    "social distancing":("1FEB2020", "1FEB2021"), 
                    "tracetogether":("1MAR2020", "1MAR2021"), 
                    "vaccination":("1AUG2020","1MAR2021")}

for policy, timeframe in policy_timeframe.items():
    sentiment_timeseries_with_cases(policy, timeframe)
    # break


circuit breaker is done!


economic measures is done!


foreign worker is done!


mask is done!


social distancing is done!


tracetogether is done!


vaccination is done!
